In [34]:
import numpy as np
import pandas as pd
from collections import OrderedDict, Counter
import itertools
import plotly.express as px

def flat_list(d_list):
    '''
    dependencies: itertools
    '''
    return list(itertools.chain.from_iterable(d_list))


def incal_create_df_incal_format(df_data_experiment, df_design_expriment):
    df = df_data_experiment.copy()
    categories_groups = df_design_expriment.values[:, 0]
    categories_subjects = list(
        filter(lambda x: ~np.isnan(x),
               (flat_list(df_design_expriment.values[:, 1:]))))

    date_time_level = pd.Series((pd.DatetimeIndex(df['Date_Time_1'])),
                                name='Date_Time_1')
    subjects_level = pd.Series(pd.Categorical(df['subjectID'],
                                              categories=categories_subjects,
                                              ordered=True),
                               name='subjectsID')
    group_level = pd.Series(pd.Categorical(df['Group'],
                                           categories=categories_groups,
                                           ordered=True),
                            name='Group')

    df = df.drop(columns=['Date_Time_1', 'subjectID', 'Group'])

    multi_index_dataframe = pd.concat(
        [date_time_level, subjects_level, group_level], axis=1)

    return pd.DataFrame(df.values,
                        index=pd.MultiIndex.from_frame(multi_index_dataframe),
                        columns=df.columns.values.tolist())

In [35]:
df_data_experiment = pd.read_csv('../csvs/InCal_format_meital.csv')
df_design_expriment = pd.read_csv('../csvs/InCal_format_Design.csv')
df = incal_create_df_incal_format(df_data_experiment, df_design_expriment)
df = df.drop(columns=['vco2', 'vh2o', 'vo2', 'xbreak', 'ybreak'])
df.sort_index(level=['Date_Time_1','subjectsID'], inplace=True)

df.head(3)


actual_allmeters  actual_foodupa  \
Date_Time_1         subjectsID Group                                       
2021-10-10 07:47:00 1.0        Control          2.584852             0.0   
                    4.0        Control          4.507522             0.0   
                    7.0        Control          7.032237             0.0   

                                        actual_pedmeters  actual_waterupa  \
Date_Time_1         subjectsID Group                                        
2021-10-10 07:47:00 1.0        Control          2.210261         0.000000   
                    4.0        Control          3.976356         0.059264   
                    7.0        Control          6.079361         0.000000   

                                        actual_wheelmeters  bodymass  \
Date_Time_1         subjectsID Group                                   
2021-10-10 07:47:00 1.0        Control            90.66000  16.32920   
                    4.0        Control           114.03480  18.27759   
                    7.0        Control             2.52539  15.35924   

                                         kcal_hr        rq  
Date_Time_1         subjectsID Group                        
2021-10-10 07:47:00 1.0        Control  0.461753  0.890226  
                    4.0        Control  0.491410  0.932738  
                    7.0        Control  0.273321  0.877051

In [36]:
df.index = df.index.droplevel(2)

In [37]:
index_datetime = df.index.get_level_values(0)
index_datetime

DatetimeIndex(['2021-10-10 07:47:00', '2021-10-10 07:47:00',
               '2021-10-10 07:47:00', '2021-10-10 07:47:00',
               '2021-10-10 07:47:00', '2021-10-10 07:47:00',
               '2021-10-10 07:47:00', '2021-10-10 07:47:00',
               '2021-10-10 07:47:00', '2021-10-10 07:47:00',
               ...
               '2021-10-17 09:07:00', '2021-10-17 09:07:00',
               '2021-10-17 09:07:00', '2021-10-17 09:07:00',
               '2021-10-17 09:07:00', '2021-10-17 09:07:00',
               '2021-10-17 09:07:00', '2021-10-17 09:07:00',
               '2021-10-17 09:07:00', '2021-10-17 09:07:00'],
              dtype='datetime64[ns]', name='Date_Time_1', length=8144, freq=None)

In [38]:
def count_custome_day_start_and_end(pd_index_datetime, shift_hour):
    dates = sorted(set(pd_index_datetime.date))
    conditiones = [ logic 
        for date in dates 
        for logic in [
          ((pd_index_datetime.hour < shift_hour) & (pd_index_datetime.date == date)), # day before
          ((pd_index_datetime.hour >= shift_hour) & (pd_index_datetime.date == date))] # day after
      ]
    langth = (len(conditiones)//2) + 1
    choices = [0, *np.repeat(range(1, langth), 2)]
    choices.pop()
    return np.select(conditiones, choices, pd_index_datetime.factorize()[0])
df['exp_days'] = count_custome_day_start_and_end(index_datetime, 8)

In [39]:
df = df.groupby(['exp_days', df.index.get_level_values(1)]).agg({
  'actual_foodupa': 'sum',
  'kcal_hr': 'sum',
  'actual_allmeters': 'mean', 
  "bodymass": 'mean',
  'actual_wheelmeters': 'mean', 
  'rq': 'mean',
  'actual_pedmeters': 'mean',
})

In [42]:
px.box(x=df.index.get_level_values(0), y=df['actual_foodupa'], template='simple_white', labels={
  'y': 'Food intake', 
  'x': 'Days of expriment'
})

In [44]:
df.to_csv('custome_days_meital_exp_data.csv')